# Module 5 Examples - Data Preparation

Building on the last assignment, we will now prepare the AirBnB data set for downstream analysis. In particular, we want to eventually use this data to get an understanding of how profitable AirBnBs really are in Chicago. Keep in mind that there are a lot of tips and tricks out there for how to most effectively use pandas, and you will discover them over time.  These examples, at times, go beyond the "base" level code, but only for the purpose of giving you ideas of what is out there.  Remember, if there is an action you want to do, there is probably a way to do it in pandas!

In [ ]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
listings = pd.read_csv('Chicago_airbnb_listings.csv')

In [ ]:
listings

## Deleting Columns

To make the columns more manageable, we will first eliminate the columns that we know we will not need later.  In the EDA, we should have found that there were two columns with no non-null values.  There's no use keeping these, so we will go ahead and delete them.

In [ ]:
listings = listings.drop(['neighbourhood_group_cleansed','calendar_updated'],axis=1) #this deletes the listed columns. An axis of 1 means columns, whereas 0 means rows
listings.shape

We can also delete other columns that are not likely going to be of use to us.  One example of this are the URL columns; they are either easily recreatable with other columns, like property ID, or they are for image data that we will not be able to use for our analysis. Let's find and eliminate these columns.

While we could delete these one at a time, wouldn't it be nice if we could find and delete all of the columns with 'url' in their title at once? To help, we notice a pattern that all of these columns have the text 'url' in the title, but we could also look for text like "https://". In the next module we will look at a more sophisticated way to code this, but for now we will take this one step at a time using methods we have already learned.

In [ ]:
col_names = listings.columns.to_list() #this gets a list of all of the columns in the dataframe
col_names_to_delete = ([col for col in col_names if "url" in col]) #this uses list comprehension to find only those column names that include the substring "url"
col_names_to_delete
listings = listings.drop(col_names_to_delete,axis=1) #the drop command can take a list of columns as well as a single column name like we used before
listings.shape

We could also choose to remove a number of other columns that probably aren't going to help us.  For example, after verifying that all of the listings have last scraped dates within a day of each other, we can remove this just to make our data look a little more manageable.  Below you will see a few other columns have been dropped. We'll probably want to remove more of these later, so this is just a first pass.

In [ ]:
listings = listings.drop(['scrape_id','last_scraped','source'],axis=1)

## Filtering

As this is an Airbnb dataset, let's say we aren't interested in any listings for hotels so let's filter out any listings for hotels. We can do this by using pandas string manipulations to identify any rows that contain the text 'hotel'.

In [ ]:
# The simple way to do this is to filter out exact values, like below:
listings[listings['property_type'] != 'Room in hotel']
#However, this can be annoying to individually find and delete for each value (as there are several different values that represent hotels). Instead, let's use string manipulation.
listings = listings[listings['property_type'].str.contains('hotel')==False] #finds all rows where the property_type column includes the text 'hotel' and filters these out of the dataframe
listings.shape

## Cleaning

### Duplicates

In [ ]:
listings[listings.duplicated()] #finds any duplicated rows. remember, by default this is interpreted as listings rows where the duplicated property is true, and is equivalent to the statement listings[listings.duplicated()==True]

Luckily, our data has no duplicates, but this is definitely something you always want to check!

### Missing Data
Note, isnull() and isna() can be used interchangeably and do the same thing

In [ ]:
listings.isnull() #this isn't a very helpful view -- let's see if how many rows have any null values
listings[listings.isna().any(axis=1)].shape #by using the .any keyword, we are checking to see if any of the values in the row are null.  .all is another keyword you can use. 

We can see that the majority of our data has at least one null value, so we probably don't want to fully delete every row with a null.  We have a few strategies we could take here, but since our EDA suggested that all of the datapoints had non-null values for vital identifiers like id and name, we'll move forward with a thresholding approach. Let's first try to get a sense of the percent of missing data in each row.

In [ ]:
counts = listings.isnull().sum(axis=1) #here, we are using the same isnull() method, but instead of just returning a boolean value like with .any or .all, we are passing that forward into a summing function that returns a count for every row
counts.value_counts(ascending=False)
counts.hist() #we can then visualize this to get a sense for the data

While highly context dependent, a general rule of thumb is that it can become harder to work with data once it's missing 20-25% of the data points, provided this represents a very small portion of the overall dataset. It doesn't look like there are too many data points that would be affected by this rule (~1%), leaving us with a workable dataset, so it seems reasonable to remove these data as a first pass.  We can always come back and remove more later if need be.

Keep in mind that this approach should not be applied blindly! If you are only interested in certain columns, using a threshold doesn't make sense; you'd want to filter based on if those columns are null instead. How to filter depends on the type of questions you are interested in answering.

In [ ]:
threshold_count = .75*len(listings.columns)
listings = listings.dropna(axis=0, thresh=threshold_count)
listings.shape

### Outliers
In the EDA, we could see that there were some extreme values, that could skew our analysis if included as is.  An example of this would be stays with a very high number of minimum nights (e.g. a couple months or more). At this point, we might conclude that these listings are no longer for short-term stays, but are for people looking for tenants. Let's go ahead and remove these listings.

In [ ]:
len(listings[listings['minimum_nights']>=45]) #check to make sure it is reasonable to get rid of this amount of data
listings = listings[listings['minimum_nights']<=45]
listings.shape
listings['minimum_nights'].describe()

### Changing Values
Finally, let's look at how we can replace values.

In [ ]:
listings['neighbourhood'].value_counts() 

Looking at the value counts above, we can see that all of these are in Chicago or surrounding areas, but there are a few different representations of 'Chicago' that we'd ideally like to merge.

Let's combine all of the 'Chicago' values first.

In [ ]:
listings['neighbourhood'] = listings['neighbourhood'].replace({'Chicago , Illinois, United States': 'Chicago, Illinois, United States', 'Chicago, United States': 'Chicago, Illinois, United States'}) #replaces the other values of Chicago so they are all in the same format
listings['neighbourhood'] = listings['neighbourhood'].replace({'Rogers Park, Il, United States': 'Rogers Park, Illinois, United States'}) #replaces a single value
listings['neighbourhood'].value_counts() 

## Transforming/Augmenting Data
Transforming and augmenting are very related topics as they are performing very similar actions, but for different reasons.

First, let's augment the data by adding a new column based on summing the values from other columns.  While we can see the total number of listings a host has and can break this down by the type, it would also be nice to see at a glance how many rooms a host has listed in the area, so we will add a column for this.

In [ ]:
listings['host_rooms_in_area'] = listings['calculated_host_listings_count_private_rooms']+ listings['calculated_host_listings_count_shared_rooms'] #this is simply creating a new column that is the sum of values in other columns
listings

We also note that we have both a column for the number of host listings in the area (`calculated_host_listings_count`) as well as the number of total host listings (`host_total_listings_count`). Since we have the local count already, let's transform the total count colums so we can easily see how many listings each host has outside of the Chicago area. To do this, we will use applying.  This example is very similar to the previous one, but can make it easier to perform more complex calculations.  There is also a more sophisticated way we can do this that we will learn about in the next module.

In [ ]:
def difference(row): #This function simply shows how to calculate the difference between two columns
	return row['host_total_listings_count']-row['calculated_host_listings_count']

listings['host_total_listings_count'] = listings.apply(difference,axis=1) #using the function to populate a new column
listings['host_total_listings_count'].value_counts(ascending=False)
listings['host_total_listings_count'][listings['host_total_listings_count']<100].hist()

From this, we can see that there is a pretty wide spread between hosts tha have very few properties and one that have a lot. But wait! Since we saw that a lot of hosts have multiple listings in the Chicago area, looking at the value counts will include duplicated hosts.  Let's de-duplicate the hosts and look at this again.

In [ ]:
unique_hosts = listings.drop_duplicates(subset=['host_id'],keep='first')
unique_hosts['host_total_listings_count'].value_counts(ascending=False)
unique_hosts['host_total_listings_count'][unique_hosts['host_total_listings_count']<100].hist()

While the distribution isn't too different, the values are!

This goes to show how EDAs and data cleaning are very iterative processes.  After making these initial preparations, the next step would be to perform another EDA and then perfrorming further cleaning and preparation as necessary.